In [5]:
%load_ext autoreload
%autoreload 2

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["ZAPIER_NLA_API_KEY"] = os.environ.get("ZAPIER_NLA_API_KEY", "")

In [7]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain, TransformChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langchain.tools.zapier.tool import ZapierNLARunAction
from langchain.utilities.zapier import ZapierNLAWrapper

In [31]:
actions = ZapierNLAWrapper().list()

In [9]:
## step 1. gmail find email

GMAIL_SEARCH_INSTRUCTIONS = "Grab the latest email from Binance"

def nla_gmail(inputs):
    action = next((a for a in actions if a["description"].startswith("Gmail: Find Email")), None)
    return {"email_data": ZapierNLARunAction(action_id=action["id"], zapier_description=action["description"], params_schema=action["params"]).run(inputs["instructions"])}

gmail_chain = TransformChain(input_variables=["instructions"], output_variables=["email_data"], transform=nla_gmail)

In [37]:
## step 2. generate draft reply

template = """You are an assisstant who drafts replies to an incoming email. Output draft reply in plain text (not JSON).

Incoming email:
{email_data}

Draft email reply:"""

prompt_template = PromptTemplate(input_variables=["email_data"], template=template)
reply_chain = LLMChain(llm=OpenAI(temperature=.7), prompt=prompt_template)

In [11]:
## finally, execute

overall_chain = SimpleSequentialChain(chains=[gmail_chain, reply_chain], verbose=True)
overall_chain.run(GMAIL_SEARCH_INSTRUCTIONS)



> Entering new SimpleSequentialChain chain...
{"from__email": "do_not_reply@mailersp1.binance.com", "from__name": "Binance", "body_plain": "\u6625\u5b63\u798f\u5229\u5927\u653e\u9001\uff01\u5e01\u5b89\u51c6\u5907\u4e8620,000 USDT\u4ee3\u5e01\u5238\u6625\u5b63\u793c\u5305\uff0c\u65b0\u8001\u7528\u6237\u5747\u53ef\u53c2\u4e0e\u6d3b\u52a8\uff0c\u8d62\u53d6\u8d85\u503c\u5956\u52b1\u3002\u6d3b\u52a8\u65f6\u95f4\uff1a2023\u5e7403\u670821\u65e508:00\u81f32023\u5e7404\u670804\u65e507:59\uff08\u4e1c\u516b\u533a\u65f6\u95f4\uff09\u3002\u6d3b\u52a8\u4e00\uff1a\u65b0", "subject": "\u6625\u5b63\u798f\u5229\u5927\u653e\u9001\uff0c20,000 USDT\u4ee3\u5e01\u5238\u60ca\u559c\u9001\u7ed9\u4f60"}

Dear Binance,

Thank you for your email regarding the Spring Festival event. We are excited to hear about the 20,000 USDT bonus that you are offering. We look forward to participating in the event, which will take place from March 21, 2023, 8:00 AM to April 4, 2023, 7:59 PM (Eastern Time).

We look forward to 

'\nDear Binance,\n\nThank you for your email regarding the Spring Festival event. We are excited to hear about the 20,000 USDT bonus that you are offering. We look forward to participating in the event, which will take place from March 21, 2023, 8:00 AM to April 4, 2023, 7:59 PM (Eastern Time).\n\nWe look forward to hearing more about the event.\n\nSincerely, \n[Your Name]'

In [16]:
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

In [47]:
def nla_gmail_draft(inputs):
    action = next((a for a in actions if a["description"].startswith("Gmail: Create Draft")), None)
    ZapierNLARunAction(action_id=action["id"], zapier_description=action["description"], params_schema=action["params"]).run(inputs)

# gmail_draft_chain = TransformChain(input_variables=["instructions"], output_variables=["email_data"], transform=nla_gmail)

In [54]:
tools = [
    Tool(
        name = "Create gmail draft",
        func=nla_gmail_draft,
        description="useful for when create a draft in gmail."
    ),
]

In [55]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm=ChatOpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent="chat-conversational-react-description", verbose=True, memory=memory)

In [56]:
agent_chain.run('write a email to project contributors, the project is a Rust telegram bot, detail the roadmap, then create draft in gmail')



> Entering new AgentExecutor chain...
{
    "action": "Create gmail draft",
    "action_input": "To: [insert email addresses of project contributors]\nSubject: Rust Telegram Bot Project Roadmap\n\nDear Project Contributors,\n\nI am writing to provide an update on the Rust Telegram Bot project and to detail our roadmap for the coming months. [Insert details of roadmap here].\n\nThank you for your continued support and contributions to this project.\n\nBest regards,\n[Your Name]"
}
Observation: None
Thought:{
    "action": "Create gmail draft",
    "action_input": "To: [insert email addresses of project contributors]\nSubject: Rust Telegram Bot Project Roadmap\n\nDear Project Contributors,\n\nI am writing to provide an update on the Rust Telegram Bot project and to detail our roadmap for the coming months. [Insert details of roadmap here].\n\nThank you for your continued support and contributions to this project.\n\nBest regards,\n[Your Name]"
}
Observation: None
Thought:My apologies f

'You should create an email draft in Gmail to the project contributors detailing the roadmap for the Rust Telegram Bot project.'